In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("AnnotatedComplete.csv")

In [3]:
df.head()

,Id,Is this a meme?,Image,Text,Image + Text,Against Women,Against Country/Region,Against Sexual Preference,Against Religion/Ethinicity,Against Celebrity/Individual,Against Brand/Industry/Product,Against Organization,Sexual Content,Violent Content,Other Reason
0,xu6kzz.jpg,Yes,Non Harmful,Harmful,Harmful,0,0,0,0,0,0,0,1,0,1
1,xtqi7z.jpg,Yes,Non Harmful,Harmful,Harmful,1,0,0,1,0,0,0,0,0,0
2,xu257y.jpg,Yes,Harmful,Harmful,Harmful,0,0,0,1,0,0,0,0,1,0
3,xu4uwg.jpg,Yes,Harmful,Non Harmful,Harmful,1,0,0,0,0,0,0,0,0,1
4,xu6u3h.jpg,Yes,Non Harmful,Harmful,Harmful,1,0,0,0,0,0,0,1,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3681 entries, 0 to 3680
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Id                              3681 non-null   object
 1   Is this a meme?                 3681 non-null   object
 2   Image                           3681 non-null   object
 3   Text                            3681 non-null   object
 4   Image + Text                    3681 non-null   object
 5   Against Women                   3681 non-null   int64 
 6   Against Country/Region          3681 non-null   int64 
 7   Against Sexual Preference       3681 non-null   int64 
 8   Against Religion/Ethinicity     3681 non-null   int64 
 9   Against Celebrity/Individual    3681 non-null   int64 
 10  Against Brand/Industry/Product  3681 non-null   int64 
 11  Against Organization            3681 non-null   int64 
 12  Sexual Content                  3681 non-null   

In [5]:
# pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib -q

In [6]:
from __future__ import print_function
import pickle
import os.path
import io
import shutil
import requests
from mimetypes import MimeTypes
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from urllib.error import HTTPError

class DriveAPI:
    global SCOPES
    folder_IDS = []
    # Define the scopes
    SCOPES = ['https://www.googleapis.com/auth/drive']
  
    def __init__(self):
        self.creds = None
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                self.creds = pickle.load(token)
  
        if not self.creds or not self.creds.valid:
  
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'client_secret_943654123571-odgkeegud532d6h4jtg8lmfnpj6vfvp8.apps.googleusercontent.com.json', SCOPES)
                self.creds = flow.run_local_server(port=0)
  
            with open('token.pickle', 'wb') as token:
                pickle.dump(self.creds, token)
  
        self.service = build('drive', 'v3', credentials=self.creds)
  
        root_folder_id = "1LJHa76BIxgpiXEmjN2RjbiBr5FXXPb6J"
        query = f"'{root_folder_id}' in parents"
        results = self.service.files().list(q=query,fields="nextPageToken, files(id, name)").execute()
        items = results.get("files", [])

        for item in items:
            self.folder_IDS.append(item['id'])
        print(len(self.folder_IDS))

  
    def FileDownload(self, file_name):
        try:
            file_id = None
            for folder in self.folder_IDS:
                query = f"'{folder}' in parents"
                results = self.service.files().list(q=query,fields="nextPageToken, files(id, name)", pageSize = 1000).execute()
                items = results.get("files", [])
                next_page_token = results.get("nextPageToken", None)

                while next_page_token:
                    query = f"'{folder}' in parents"
                    results = self.service.files().list(q=query,fields="nextPageToken, files(id, name)", pageToken=next_page_token, pageSize = 1000).execute()
                    items.extend(results.get("files", []))
                    next_page_token = results.get("nextPageToken", None)

                for item in items:
                    if item['name'] == file_name:
                        file_id = item['id']
                        break
    
                if file_id:
                    request = self.service.files().get_media(fileId=file_id)
                    try:
                        file = request.execute()
                        with open(os.path.join("Memes", file_name), 'wb') as f:
                            f.write(file)
                        print("File Downloaded")
                        return True
                    except:
                        print("Something went wrong.")
                        return False
                    break
            if file_id is None:
                print(F'File with name "{file_name}" not found in the folder')
                return False
            return True
        except HTTPError as error:
            print(F'An error occurred: {error}')
            return False
 

In [7]:
import os
import csv

def search_csv_files(search_id, folder_path = 'subreddits'):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                with open(file_path, newline='') as csvfile:
                    reader = csv.DictReader(csvfile)
                    for row in reader:
                        if row['id'] == search_id:
                            return row
    return None

def Copy_search_csv_files(search_id, folder_path = 'subreddits'):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                with open(file_path, newline='') as csvfile:
                    reader = csv.DictReader(csvfile)
                    for row in reader:
                        if row['MemeID'] == search_id:
                            return row
    return None


In [8]:
meta_data = pd.DataFrame({'id': [], 'title': [], 'score': [], 'upvote-ratio':[], 'url':[], 'num_comments': [], 'created': [], 'over18': [] })
comment_score = pd.DataFrame({'MemeID': [], 'CommentScore': []})

In [10]:
obj = DriveAPI()
for index, data_point in df.iterrows():
    print(index)
    if data_point['Is this a meme?'] == 'Yes' and data_point['Image']!='Mark For Review' and data_point['Text']!='Mark For Review' and data_point['Image + Text']!='Mark For Review':
        if data_point['Id'] not in os.listdir("Memes"):
            obj.FileDownload(data_point['Id'])
        else:
            print("Present")
        # if 0:
        #     continue
        # else:
        #     meta_data_row = search_csv_files(data_point['Id'].split(".")[0])
        #     comment_score_row = Copy_search_csv_files(data_point['Id'].split(".")[0], 'commentScores')

        #     if not meta_data_row or not comment_score_row:
        #         print("missing"+str(data_point['Id']))
        #         continue
        #     meta_data_row = {k: v for k, v in meta_data_row.items() if k in meta_data.columns}
        #     meta_data = meta_data.append(meta_data_row, ignore_index=True)

        #     # if not comment_score_row:
        #     #     print("missing"+str(data_point['Id']))
        #     #     continue

        #     comment_score_row = {k: v for k, v in comment_score_row.items() if k in comment_score.columns}
        #     comment_score = comment_score.append(comment_score_row, ignore_index=True)
                      

62
0
Present
1
Present
2
Present
3
Present
4
Present
5
Present
6
Present
7
Present
8
Present
9
Present
10
Present
11
Present
12
Present
13
Present
14
Present
15
Present
16
Present
17
Present
18
Present
19
Present
20
Present
21
Present
22
Present
23
Present
24
Present
25
Present
26
Present
27
Present
28
Present
29
Present
30
Present
31
Present
32
Present
33
Present
34
Present
35
Present
36
Present
37
Present
38
Present
39
Present
40
Present
41
Present
42
Present
43
Present
44
Present
45
Present
46
Present
47
Present
48
Present
49
Present
50
Present
51
Present
52
Present
53
Present
54
Present
55
Present
56
Present
57
Present
58
Present
59
Present
60
Present
61
Present
62
Present
63
Present
64
Present
65
Present
66
Present
67
Present
68
Present
69
Present
70
Present
71
Present
72
Present
73
Present
74
Present
75
Present
76
Present
77
Present
78
Present
79
Present
80
Present
81
Present
82
Present
83
Present
84
Present
85
Present
86
Present
87
Present
88
Present
89
Present
90
Present
91
Pre

In [39]:
# meta_data.to_csv('meta_data.csv')
# comment_score.to_csv('comment_score.csv')

In [40]:
meta_data.drop_duplicates(inplace=True)
comment_score.drop_duplicates(inplace=True)

In [41]:
import ast

comment_score['%PositiveComments'] = [ast.literal_eval(x)[0] for x in comment_score.CommentScore.values.tolist()]
comment_score['%NeutralComments'] = [ast.literal_eval(x)[1] for x in comment_score.CommentScore.values.tolist()]
comment_score['%NegativeComments'] = [ast.literal_eval(x)[2] for x in comment_score.CommentScore.values.tolist()]

In [42]:
# comment_score = comment_score.drop(['CommentScore'], axis = 1)

In [43]:
comment_score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3093 entries, 0 to 3151
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MemeID             3093 non-null   object 
 1   CommentScore       3093 non-null   object 
 2   %PositiveComments  3093 non-null   float64
 3   %NeutralComments   3093 non-null   float64
 4   %NegativeComments  3093 non-null   float64
dtypes: float64(3), object(2)
memory usage: 145.0+ KB


In [44]:
meta_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3093 entries, 0 to 3151
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            3093 non-null   object
 1   title         3093 non-null   object
 2   score         3093 non-null   object
 3   upvote-ratio  3093 non-null   object
 4   url           3093 non-null   object
 5   num_comments  3093 non-null   object
 6   created       3093 non-null   object
 7   over18        3093 non-null   object
dtypes: object(8)
memory usage: 217.5+ KB


In [109]:
# IDDF = meta_data[~(meta_data['id'].isin(comment_score['MemeID'].values))]
# IDDF['id'].to_csv("ID.csv")

In [52]:
meta_data.rename(columns = {'id':'Id'}, inplace = True)
comment_score.rename(columns = {'MemeID':'Id'}, inplace = True)

In [53]:
df['Id'] = df['Id'].apply(lambda x: x.replace(x,x.split(".")[0] ))

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3681 entries, 0 to 3680
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Id                              3681 non-null   object
 1   Is this a meme?                 3681 non-null   object
 2   Image                           3681 non-null   object
 3   Text                            3681 non-null   object
 4   Image + Text                    3681 non-null   object
 5   Against Women                   3681 non-null   int64 
 6   Against Country/Region          3681 non-null   int64 
 7   Against Sexual Preference       3681 non-null   int64 
 8   Against Religion/Ethinicity     3681 non-null   int64 
 9   Against Celebrity/Individual    3681 non-null   int64 
 10  Against Brand/Industry/Product  3681 non-null   int64 
 11  Against Organization            3681 non-null   int64 
 12  Sexual Content                  3681 non-null   

In [55]:
df.head()

,Id,Is this a meme?,Image,Text,Image + Text,Against Women,Against Country/Region,Against Sexual Preference,Against Religion/Ethinicity,Against Celebrity/Individual,Against Brand/Industry/Product,Against Organization,Sexual Content,Violent Content,Other Reason
0,xu6kzz,Yes,Non Harmful,Harmful,Harmful,0,0,0,0,0,0,0,1,0,1
1,xtqi7z,Yes,Non Harmful,Harmful,Harmful,1,0,0,1,0,0,0,0,0,0
2,xu257y,Yes,Harmful,Harmful,Harmful,0,0,0,1,0,0,0,0,1,0
3,xu4uwg,Yes,Harmful,Non Harmful,Harmful,1,0,0,0,0,0,0,0,0,1
4,xu6u3h,Yes,Non Harmful,Harmful,Harmful,1,0,0,0,0,0,0,1,0,0


In [56]:
print(set(meta_data['Id'])-set(comment_score['Id']))

set()


In [57]:
merged_df = pd.merge(meta_data, comment_score, on='Id', how='outer')
merged_df = pd.merge(merged_df, df, on='Id', how='outer')
merged_df.head()

,Id,title,score,upvote-ratio,url,num_comments,created,over18,CommentScore,%PositiveComments,...,Against Women,Against Country/Region,Against Sexual Preference,Against Religion/Ethinicity,Against Celebrity/Individual,Against Brand/Industry/Product,Against Organization,Sexual Content,Violent Content,Other Reason
0,xu6kzz,This one is so dried it’s crunchy,158,0.92,https://i.redd.it/8j4s5h3a1ir91.jpg,6,2022-10-03 07:38:14,False,"[0.16666666666666666, 0.8333333333333334, 0.0]",0.166667,...,0,0,0,0,0,0,0,1,0,1
1,xtqi7z,I guess only half of the human race is favored...,1725,0.95,https://i.redd.it/i8lmbatymer91.png,231,2022-10-02 20:12:43,False,"[0.04950495049504951, 0.9504950495049505, 0.0]",0.049505,...,1,0,0,1,0,0,0,0,0,0
2,xu257y,What if they are herbivores?,408,0.92,https://i.redd.it/kd6ggcyh0hr91.jpg,40,2022-10-03 04:12:04,False,"[0.08108108108108109, 0.918918918918919, 0.0]",0.081081,...,0,0,0,1,0,0,0,0,1,0
3,xu4uwg,It’s snowing!,276,0.97,https://i.redd.it/hn01xkgsmhr91.jpg,25,2022-10-03 06:17:06,False,"[0.08695652173913043, 0.9130434782608695, 0.0]",0.086957,...,1,0,0,0,0,0,0,0,0,1
4,xu6u3h,Not from Facebook but ripped from TikTok,218,0.93,https://i.redd.it/dv43t4fd3ir91.jpg,11,2022-10-03 07:49:57,False,"[0.09090909090909091, 0.9090909090909091, 0.0]",0.090909,...,1,0,0,0,0,0,0,1,0,0


In [58]:
merged_df = merged_df.drop(['Is this a meme?'], axis=1)

In [59]:
merged_df = merged_df.dropna()
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3093 entries, 0 to 3092
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Id                              3093 non-null   object 
 1   title                           3093 non-null   object 
 2   score                           3093 non-null   object 
 3   upvote-ratio                    3093 non-null   object 
 4   url                             3093 non-null   object 
 5   num_comments                    3093 non-null   object 
 6   created                         3093 non-null   object 
 7   over18                          3093 non-null   object 
 8   CommentScore                    3093 non-null   object 
 9   %PositiveComments               3093 non-null   float64
 10  %NeutralComments                3093 non-null   float64
 11  %NegativeComments               3093 non-null   float64
 12  Image                           30

In [60]:
merged_df.to_csv('MergedComplete.csv')